In [3]:
import math
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
import argparse
import os
import tqdm
import inspect
import logging

from models.teacher import Teacher
from models.configuration_teacher import TeacherConfig
from data import CoTDataset, CoTDataCollator, extract_answer

from utils import get_sep_position

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
logging.disable(logging.WARNING) # disable WARNING, INFO and DEBUG logging everywhere

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def save_model(model, tokenizer, model_dir):
    print ('saving', model_dir)
    os.makedirs(model_dir, exist_ok=True)
    model.save_pretrained(model_dir)
    tokenizer.save_pretrained(model_dir)

@torch.no_grad()
def evaluate(dataloader, tokenizer, ctx, teacher, max_new_tokens):
    teacher.eval()
    total_instances = 0
    total_tokens = 0
    total_correct = 0
    total_correct_tokens = 0
    total_loss = 0
    for batch in tqdm.tqdm(dataloader):
        input_ids_all = batch['input_ids_all'].to(device)
        labels = batch['labels_all'].to(device)
        # Remove answer part
        sep_positions = get_sep_position(input_ids_all, tokenizer.eos_token_id)
        input_ids = input_ids_all[:, :sep_positions.max()+1]
        batch_size = input_ids.shape[0]
        with ctx:
            outputs = teacher.compute_loss(input_ids=input_ids_all, labels=labels)
        total_loss += outputs.total_loss.item()
        total_correct_tokens += outputs.total_correct.item()
        total_tokens += outputs.total_tokens
        total_instances += batch_size

        # Generate
        beam_output = teacher.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
        )
        # Evaluate
        #import pdb; pdb.set_trace()
        for i, (input_ids_all_i, beam_output_i) in enumerate(zip(input_ids_all, beam_output)):
            sep_position = sep_positions[i].item()
            tgt = input_ids_all_i[sep_position+1:]
            tgt_text = tokenizer.decode(tgt, skip_special_tokens=True)
            ans = extract_answer(tgt_text)
            pred_text = tokenizer.decode(beam_output_i[0][sep_position+1:], skip_special_tokens=True)
            pred_ans = extract_answer(pred_text)
            if ans == pred_ans:
                total_correct += 1
            if i == 0:
                print (f'Input: {tokenizer.decode(input_ids_all_i[:sep_position], skip_special_tokens=True)}')
                print (f'Target: {tgt_text}')
                print (f'Predicted: {pred_text}')
                print ('')
    accuracy = total_correct / total_instances
    token_accuracy = total_correct_tokens / total_tokens
    loss = total_loss / total_tokens
    ppl = math.exp(loss)
    return accuracy, token_accuracy, ppl


    



2024-01-10 10:49:08.880576: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-10 10:49:08.902826: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-10 10:49:09.244866: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
teacher_trainer_args=dict(
    debug=False,
    train_path="../data/4_by_4_mult/train.txt",
    val_path="../data/4_by_4_mult/valid.txt",
    save_model="../train_models/4_by_4_mult/gpt2/teacher",
    max_new_tokens=128,
    base_model='gpt2',
    epochs=1,
    batch_size=16,
    lr=5e-5,
    max_grad_norm=1.0,
    
)

from types import SimpleNamespace

args = SimpleNamespace(**teacher_trainer_args)

# parser = argparse.ArgumentParser()
# parser.add_argument('--train_path', type=str, required=True)
# parser.add_argument('--val_path', type=str, required=True)
# parser.add_argument('--save_model', type=str, required=True)
# parser.add_argument('--max_new_tokens', type=int, default=128)
# parser.add_argument('--base_model', type=str, default='gpt2')
# parser.add_argument('--epochs', type=int, default=1)
# parser.add_argument('--batch_size', type=int, default=5e-5)
# parser.add_argument('--lr', type=float, default=5e-5)
# parser.add_argument('--max_grad_norm', type=float, default=1.0)
# args = parser.parse_args()

In [8]:
print (args)

dtype = 'float32'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ctx = torch.amp.autocast(device_type='cuda', dtype=ptdtype)
print (ptdtype, dtype, device)

# Create Student 
config = TeacherConfig(base_model=args.base_model)
teacher = Teacher(config).to(device).to(ptdtype)

# Load data
tokenizer = teacher.tokenizer
collate_fn = CoTDataCollator(tokenizer)
train_dataset = CoTDataset(tokenizer, args.train_path, 1024)
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, collate_fn=collate_fn, shuffle=True)
val_dataset = CoTDataset(tokenizer, args.val_path, 1024)
val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, collate_fn=collate_fn, shuffle=False)

# Create Optimizer
trainable_params = list(teacher.parameters())
use_fused = 'fused' in inspect.signature(torch.optim.AdamW).parameters
extra_args = dict(fused=True) if use_fused else dict()
optimizer = torch.optim.AdamW(trainable_params, lr=args.lr, **extra_args)



namespace(debug=False, train_path='../data/4_by_4_mult/train.txt', val_path='../data/4_by_4_mult/valid.txt', save_model='../train_models/4_by_4_mult/gpt2/teacher', max_new_tokens=128, base_model='gpt2', epochs=1, batch_size=16, lr=5e-05, max_grad_norm=1.0)
torch.float32 float32 cuda
Creating features from dataset file at ../data/4_by_4_mult/train.txt
tgt_avg:  49.0
src_avg:  10.0
ratios:  0.20408163265306123
tgt_avg:  13.0
src_avg:  10.0
ratios:  0.7692307692307693
 1 3 3 8 * 5 1 0 5 <|endoftext|> 5 5 6 1 4 + 0 1 3 3 8 0 ( 5 6 9 4 2 1 ) + 0 0 0 0 0 0 0 ( 5 6 9 4 2 1 0 ) + 0 0 0 5 5 6 1 4 <|endoftext|> #### 5 6 9 9 7 7 1 4 <|endoftext|>
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 642, 642, 718, 352, 604, 1343, 657, 352, 513, 513, 807, 657, 357, 642, 718, 860, 604, 362, 352, 1267, 1343, 657, 657, 657, 657, 657, 657, 657, 357, 642, 718, 860, 604, 362, 352, 657, 1267, 1343, 657, 657, 657, 642, 642, 718, 352, 604, 220, 50256]
[-100, -100, -100, -100, -100, -100, -100,

In [19]:
teacher.train()

# Train
step = 0
for epoch in range(args.epochs):
    print(f"Epoch {epoch}")
    teacher.train()
    for batch in tqdm.tqdm(train_dataloader):
        input_ids = batch['input_ids_all'].to(device)
        labels = batch['labels_all'].to(device)
        with ctx:
            outputs = teacher.compute_loss(input_ids=input_ids, labels=labels)
#         break
        loss = outputs.loss
        token_accuracy = outputs.token_accuracy.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(trainable_params, args.max_grad_norm)
        optimizer.step()
        optimizer.zero_grad()
        ppl = loss.exp().item()
        if step % 100 == 0:
            print (f"Step: {step}. PPL: {ppl}. Token Accuracy: {token_accuracy}")
        step += 1
#     break
    accuracy, token_accuracy, ppl = evaluate(val_dataloader, tokenizer, ctx, teacher, args.max_new_tokens)
    print (f'Val. PPL: {ppl}; Accuracy: {accuracy}; Token Accuracy: {token_accuracy}.')
    teacher.save_pretrained(os.path.join(args.save_model, f'checkpoint_{epoch}'))

Epoch 0


  0%|                                       | 1/50500 [00:00<1:48:34,  7.75it/s]

Step: 0. PPL: 37.15435028076172. Token Accuracy: 0.20104166865348816


  0%|                                     | 103/50500 [00:07<1:01:19, 13.70it/s]

Step: 100. PPL: 3.1745927333831787. Token Accuracy: 0.6177083253860474


...

Input:  0 8 3 6 * 0 0 3 1 
Target:  0 0 0 0 0 + 0 0 0 0 0 0 ( 0 0 0 0 0 0 ) + 0 0 0 4 1 9 1 ( 0 0 0 4 1 9 1 ) + 0 0 0 0 8 3 6 0  #### 0 0 0 4 9 2 8 0 
Predicted:  0 0 0 0 0 + 0 0 0 0 0 0 ( 0 0 0 0 0 0 ) + 0 0 0 4 1 9 1 ( 0 0 0 4 1 9 1 ) + 0 0 0 0 8 3 6 0  #### 0 0 0 4 9 2 8 0 



 98%|██████████████████████████████████████████▎| 62/63 [00:13<00:00,  4.59it/s]

Input:  7 2 3 3 * 6 2 9 6 
Target:  2 6 9 9 1 + 0 4 5 6 6 0 ( 2 0 5 6 8 0 ) + 0 0 3 4 9 9 2 ( 2 0 8 0 8 0 3 ) + 0 0 0 2 6 9 9 1  #### 2 0 8 2 4 0 3 2 
Predicted:  2 6 9 9 1 + 0 4 5 6 6 0 ( 2 0 5 6 8 0 ) + 0 0 3 4 9 9 2 ( 2 0 8 0 8 0 3 ) + 0 0 0 2 6 9 9 1  #### 2 0 8 2 4 0 3 2 



100%|███████████████████████████████████████████| 63/63 [00:13<00:00,  4.57it/s]

Input:  5 7 5 2 * 2 2 7 3 
Target:  0 5 1 5 0 + 0 0 5 1 5 0 ( 0 5 6 6 5 0 ) + 0 0 5 2 0 8 1 ( 0 5 1 9 5 8 1 ) + 0 0 0 5 2 7 7 0  #### 0 5 1 4 8 5 9 0 
Predicted:  0 5 1 5 0 + 0 0 5 1 5 0 ( 0 5 6 6 5 0 ) + 0 0 5 2 0 8 1 ( 0 5 1 9 5 8 1 ) + 0 0 0 5 2 7 7 0  #### 0 5 1 4 8 5 9 0 

Val. PPL: 1.000001479784104; Accuracy: 1.0; Token Accuracy: 1.0.
Saving to ../train_models/4_by_4_mult/gpt2/teacher/checkpoint_0


In [18]:
print(outputs.attentions.shape)

AttributeError: 'NoneType' object has no attribute 'shape'

In [5]:
import math
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
import argparse
import os
import sys
import inspect
import tqdm
import logging
import random

from data import CoTDataset, CoTDataCollator, extract_answer
from models.teacher import Teacher
from models.student import Student
from models.configuration_student import StudentConfig
from utils import get_sep_position

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

random.seed(1234)
torch.manual_seed(1234)
logging.disable(logging.WARNING) # disable WARNING, INFO and DEBUG logging everywhere

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

@torch.no_grad()
def evaluate(dataloader, tokenizer, ctx, teacher, student, delta, subset, max_new_tokens):
    total_instances = 0
    total_tokens = 0
    total_correct = 0
    total_correct_tokens = 0
    total_loss = 0
    for batch in tqdm.tqdm(dataloader):
        input_ids_all = batch['input_ids_all'].to(device)
        input_ids_nocot = batch['input_ids_nocot'].to(device)
        labels_nocot = batch['labels_nocot'].to(device)
        batch_size = input_ids_nocot.shape[0]
        with ctx:
            teacher_states = teacher.extract_states(input_ids=input_ids_all, delta=delta, subset=subset)
            outputs = student.compute_loss(input_ids=input_ids_nocot, labels=labels_nocot, teacher_states=teacher_states)
            loss = outputs.loss
            token_accuracy = outputs.token_accuracy.item()
        total_loss += outputs.total_loss.item()
        total_correct_tokens += outputs.total_correct.item()
        total_tokens += outputs.total_tokens
        total_instances += batch_size

        # Generate
        with ctx:
            beam_output = student.generate(
                input_ids=input_ids_nocot,
                teacher_states=teacher_states,
                max_new_tokens=max_new_tokens,
            )

        # Evaluate
        sep_positions = get_sep_position(input_ids_all, tokenizer.eos_token_id)
        for i, (input_ids_all_i, beam_output_i) in enumerate(zip(input_ids_all, beam_output)):
            sep_position = sep_positions[i].item()
            tgt = input_ids_all_i[sep_position+1:]
            tgt_text = tokenizer.decode(tgt, skip_special_tokens=True)
            ans = extract_answer(tgt_text)
            pred_text = tokenizer.decode(beam_output_i[0][sep_position+1:], skip_special_tokens=True)
            pred_ans = extract_answer(pred_text)
            if ans == pred_ans:
                total_correct += 1
            if i == 0:
                print (f'Input: {tokenizer.decode(input_ids_all_i[:sep_position], skip_special_tokens=True)}')
                print (f'Target: {tgt_text}')
                print (f'Predicted: {pred_text}')
                print ('')
    accuracy = total_correct / total_instances
    token_accuracy = total_correct_tokens / total_tokens
    loss = total_loss / total_tokens
    ppl = math.exp(loss)
    return accuracy, token_accuracy, ppl


    

In [35]:
mind_reading_student_trainer_args=dict(
    debug=False,
    train_path="../data/4_by_4_mult/train.txt",
    val_path="../data/4_by_4_mult/valid.txt",
    teacher="../train_models/4_by_4_mult/gpt2/teacher/checkpoint_0",
    save_model="../train_models/4_by_4_mult/gpt2/student_initial",
    base_model='gpt2',
    epochs=1,
    batch_size=16,
    lr=5e-5,
    max_new_tokens=128,
    delta='dynamic',
    max_grad_norm=1.0,
    subset='diagonal',
    
)

from types import SimpleNamespace

args = SimpleNamespace(**mind_reading_student_trainer_args)

In [36]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--teacher', type=str, required=True)
# parser.add_argument('--delta', type=str, required=True)
# parser.add_argument('--train_path', type=str, required=True)
# parser.add_argument('--val_path', type=str, required=True)
# parser.add_argument('--save_model', type=str, required=True)
# parser.add_argument('--max_new_tokens', type=int, default=128)
# parser.add_argument('--base_model', type=str, default='gpt2')
# parser.add_argument('--epochs', type=int, default=5)
# parser.add_argument('--batch_size', type=int, default=32)
# parser.add_argument('--lr', type=float, default=5e-5)
# parser.add_argument('--max_grad_norm', type=float, default=1.0)
# parser.add_argument('--subset', type=str, choices=['diagonal', 'last_column', 'top_row', 'bottom_row', 'first_column'], default='diagonal')
# args = parser.parse_args()

# print (args)

dtype = 'float32'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ctx = torch.amp.autocast(device_type='cuda', dtype=ptdtype)
print (ptdtype, dtype, device)

# Create Student
config = StudentConfig(base_model=args.base_model)
student = Student(config).to(device).to(ptdtype)

# Load Teacher
teacher = Teacher.from_pretrained(args.teacher).to(device).to(ptdtype)

# Load data
tokenizer = teacher.tokenizer
collate_fn = CoTDataCollator(tokenizer)
train_dataset = CoTDataset(tokenizer, args.train_path, 1024)
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, collate_fn=collate_fn, shuffle=True)
val_dataset = CoTDataset(tokenizer, args.val_path, 1024)
val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, collate_fn=collate_fn, shuffle=False)

# Create Optimizer
trainable_params = list(student.parameters())
use_fused = 'fused' in inspect.signature(torch.optim.AdamW).parameters
extra_args = dict(fused=True) if use_fused else dict()
optimizer = torch.optim.AdamW(trainable_params, lr=args.lr, **extra_args)

teacher.eval()
student.eval() # to turn off dropout

for p in teacher.parameters():
    p.requires_grad = False




torch.float32 float32 cuda
Creating features from dataset file at ../data/4_by_4_mult/train.txt
tgt_avg:  49.0
src_avg:  10.0
ratios:  0.20408163265306123
tgt_avg:  13.0
src_avg:  10.0
ratios:  0.7692307692307693
 1 3 3 8 * 5 1 0 5 <|endoftext|> 5 5 6 1 4 + 0 1 3 3 8 0 ( 5 6 9 4 2 1 ) + 0 0 0 0 0 0 0 ( 5 6 9 4 2 1 0 ) + 0 0 0 5 5 6 1 4 <|endoftext|> #### 5 6 9 9 7 7 1 4 <|endoftext|>
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 642, 642, 718, 352, 604, 1343, 657, 352, 513, 513, 807, 657, 357, 642, 718, 860, 604, 362, 352, 1267, 1343, 657, 657, 657, 657, 657, 657, 657, 357, 642, 718, 860, 604, 362, 352, 657, 1267, 1343, 657, 657, 657, 642, 642, 718, 352, 604, 220, 50256]
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 1303, 21017, 642, 718, 860, 860, 767, 767, 352, 604, 220, 50256]
[352, 513, 513, 807, 1635, 642, 352, 657, 642, 220, 50256, 1303, 21017, 642, 718, 860, 860, 767, 767, 352, 604, 220, 50256]
 1 3 3 8 * 5 1 0 5 <|endoftext|> #### 5 6 9

In [37]:
teacher

Teacher(
  (base_model): GPT2LMHeadImplicitModel(
    (transformer): GPT2ImplicitModel(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Co

In [40]:
teacher.base_model

GPT2LMHeadImplicitModel(
  (transformer): GPT2ImplicitModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (res

In [75]:
def compute_positions_to_extract_per_layer(subset, delta, first_sep_positions, second_sep_positions):
    batch_size = first_sep_positions.shape[0]
    positions_to_extract_per_layer = first_sep_positions.new_zeros(batch_size, teacher.num_layers).long()
    layer_ids = torch.arange(start=0, end=teacher.num_layers).to(first_sep_positions.device)
    for batch_id in range(batch_size):
        first_position_to_extract = first_sep_positions[batch_id]
        last_position_to_extract = second_sep_positions[batch_id]
        if subset == 'diagonal':
            if delta == 'dynamic': # determine actual delta
                delta = (last_position_to_extract - first_position_to_extract) / (teacher.num_layers - 1)
        elif subset == 'first_column' or subset == 'last_column':
            delta = 0
        else:
            assert subset == 'last_column', subset
            delta = 0
            first_position_to_extract = last_position_to_extract
        positions_to_extract = torch.round(first_position_to_extract + layer_ids * delta)
        positions_to_extract = positions_to_extract.clamp(max=last_position_to_extract)
        positions_to_extract_per_layer[batch_id] = positions_to_extract
    return positions_to_extract_per_layer

In [97]:
from utils import get_sep_position

for batch in tqdm.tqdm(val_dataloader):
        input_ids_all = batch['input_ids_all'].to(device)
        first_sep_positions = get_sep_position(input_ids_all, teacher.tokenizer.eos_token_id, skip=0)
        second_sep_positions = get_sep_position(input_ids_all, teacher.tokenizer.eos_token_id, skip=1)
        print(compute_positions_to_extract_per_layer('diagonal', 'dynamic', first_sep_positions, second_sep_positions)[:,2].view(-1, 1, 1).expand(-1, -1, teacher.hidden_size).shape)
        input_ids_all = input_ids_all[:, :second_sep_positions.max()+1]
        print(input_ids_all.shape)
        input_ids_nocot = batch['input_ids_nocot'].to(device)
        labels_nocot = batch['labels_nocot'].to(device)
        with ctx:
            with torch.no_grad():
                teacher_states = teacher.base_model(input_ids=input_ids_all, output_hidden_states=True)
                hidden_states = teacher_states.hidden_states[:-1]
                for i, hidden_state in enumerate(hidden_states):
                    print(hidden_state.shape) # torch.Size([16, 59, 768])
                    z = hidden_state.gather(1, compute_positions_to_extract_per_layer('diagonal', 'dynamic', first_sep_positions, second_sep_positions)[:,i].view(-1, 1, 1).expand(-1, -1, teacher.hidden_size)).squeeze(1)
                    print(z.shape) # torch.Size([16, 768])
                    
        break

  0%|                                                    | 0/63 [00:00<?, ?it/s]

torch.Size([16, 1, 768])
torch.Size([16, 59])
torch.Size([16, 59, 768])
torch.Size([16, 768])
torch.Size([16, 59, 768])
torch.Size([16, 768])
torch.Size([16, 59, 768])
torch.Size([16, 768])
torch.Size([16, 59, 768])
torch.Size([16, 768])
torch.Size([16, 59, 768])
torch.Size([16, 768])
torch.Size([16, 59, 768])
torch.Size([16, 768])
torch.Size([16, 59, 768])
torch.Size([16, 768])
torch.Size([16, 59, 768])
torch.Size([16, 768])
torch.Size([16, 59, 768])
torch.Size([16, 768])
torch.Size([16, 59, 768])
torch.Size([16, 768])
torch.Size([16, 59, 768])
torch.Size([16, 768])
torch.Size([16, 59, 768])
torch.Size([16, 768])


In [77]:
teacher_states.hidden_states[:-1][0].shape

torch.Size([16, 59, 768])

In [ ]:
# Train
step = 0
for epoch in range(args.epochs):
    print(f"Epoch {epoch}")

    for batch in tqdm.tqdm(train_dataloader):
        input_ids_all = batch['input_ids_all'].to(device)
        input_ids_nocot = batch['input_ids_nocot'].to(device)
        labels_nocot = batch['labels_nocot'].to(device)
        with ctx:
            with torch.no_grad():
                teacher_states = teacher.extract_states(input_ids=input_ids_all, delta=args.delta, subset=args.subset)
            outputs = student.compute_loss(input_ids=input_ids_nocot, labels=labels_nocot, teacher_states=teacher_states)
        loss = outputs.loss
        token_accuracy = outputs.token_accuracy.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(trainable_params, args.max_grad_norm)
        optimizer.step()
        optimizer.zero_grad()
        ppl = loss.exp().item()
        if step % 100 == 0:
            print (f"Step: {step}. PPL: {ppl}. Token Accuracy: {token_accuracy}")
            sys.stdout.flush()
        step += 1
    accuracy, token_accuracy, ppl = evaluate(val_dataloader, tokenizer, ctx, teacher, student, args.delta, args.subset, args.max_new_tokens)
    print (f'Val. PPL: {ppl}; Accuracy: {accuracy}; Token Accuracy: {token_accuracy}.')
    student.save_pretrained(os.path.join(args.save_model, f'checkpoint_{epoch}'))

In [6]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
import argparse
import os
import inspect
import tqdm
import logging
import random
import torch.nn as nn

from data import CoTDataset, CoTDataCollator
from models.teacher import Teacher
from models.emulator import Emulator
from models.configuration_emulator import EmulatorConfig


torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
random.seed(1234)
torch.manual_seed(1234)
logging.disable(logging.WARNING)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

@torch.no_grad()
def evaluate(dataloader, tokenizer, ctx, teacher, emulator, delta, subset):
    total_instances = 0
    total_loss = 0
    for batch in tqdm.tqdm(dataloader):
        #import pdb; pdb.set_trace()
        input_ids_cot = batch['input_ids_cot'].to(device)
        batch_size = input_ids_cot.shape[0]
        with ctx:
            teacher_states = teacher.extract_states(input_ids=input_ids_cot, delta=delta, subset=subset)
            outputs = emulator.compute_loss(input_ids=input_ids_cot, teacher_states=teacher_states)
            loss = outputs.loss
        total_loss += outputs.total_loss.item()
        total_instances += batch_size

    loss = total_loss / total_instances
    return loss


    

In [7]:
thought_emulator_trainer_args=dict(
    debug=False,
    train_path="../data/4_by_4_mult/train.txt",
    val_path="../data/4_by_4_mult/valid.txt",
    teacher="../train_models/4_by_4_mult/gpt2/teacher/checkpoint_0",
    save_model="../train_models/4_by_4_mult/gpt2/emulator_initial",
    base_model='gpt2',
    epochs=1,
    batch_size=16,
    lr=5e-5,
    max_new_tokens=128,
    delta='dynamic',
    max_grad_norm=1.0,
    subset='diagonal',
    mixture_size=1,
    
)

from types import SimpleNamespace

args = SimpleNamespace(**thought_emulator_trainer_args)

In [8]:

# parser = argparse.ArgumentParser()
# parser.add_argument('--teacher', type=str, required=True)
# parser.add_argument('--delta', type=str, required=True)
# parser.add_argument('--train_path', type=str, required=True)
# parser.add_argument('--val_path', type=str, required=True)
# parser.add_argument('--save_model', type=str, required=True)
# parser.add_argument('--base_model', type=str, default='gpt2')
# parser.add_argument('--epochs', type=int, default=5)
# parser.add_argument('--batch_size', type=int, default=32)
# parser.add_argument('--lr', type=float, default=5e-5)
# parser.add_argument('--max_grad_norm', type=float, default=1.0)
# parser.add_argument('--subset', type=str, choices=['diagonal', 'last_column', 'top_row', 'bottom_row', 'first_column'], default='diagonal')
# parser.add_argument('--mixture_size', type=int, default=1)
# args = parser.parse_args()

# print (args)
dtype = 'float32'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ctx = torch.amp.autocast(device_type='cuda', dtype=ptdtype)
print (ptdtype, dtype, device)

# Create Emulator
config = EmulatorConfig(base_model=args.base_model, mixture_size=args.mixture_size)
emulator = Emulator(config).to(device).to(ptdtype)

# Load Teacher
teacher = Teacher.from_pretrained(args.teacher).to(device).to(ptdtype)

# Load data
tokenizer = teacher.tokenizer
collate_fn = CoTDataCollator(tokenizer)
train_dataset = CoTDataset(tokenizer, args.train_path, 1024)
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, collate_fn=collate_fn, shuffle=True)
val_dataset = CoTDataset(tokenizer, args.val_path, 1024)
val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, collate_fn=collate_fn, shuffle=False)

# Create Optimizer
trainable_params = list(emulator.parameters())
use_fused = 'fused' in inspect.signature(torch.optim.AdamW).parameters
extra_args = dict(fused=True) if use_fused else dict()
optimizer = torch.optim.AdamW(trainable_params, lr=args.lr, **extra_args)

teacher.eval()
emulator.eval() # to turn off dropout

for p in teacher.parameters():
    p.requires_grad = False

# Train
step = 0
for epoch in range(args.epochs):
    print(f"Epoch {epoch}")

    for batch in tqdm.tqdm(train_dataloader):
        #import pdb; pdb.set_trace()
        input_ids_cot = batch['input_ids_cot'].to(device)
        input_ids_nocot = batch['input_ids_nocot'].to(device)
        with ctx:
            with torch.no_grad():
                teacher_states = teacher.extract_states(input_ids=input_ids_cot, delta=args.delta, subset=args.subset)
            outputs = emulator.compute_loss(input_ids=input_ids_nocot, teacher_states=teacher_states)
        loss = outputs.loss

        loss.backward()
        torch.nn.utils.clip_grad_norm_(trainable_params, args.max_grad_norm)
        optimizer.step()
        optimizer.zero_grad()
        if step % 100 == 0:
            print (f"Step: {step}. Loss: {loss}.")
        step += 1
    loss = evaluate(val_dataloader, tokenizer, ctx, teacher, emulator, args.delta, args.subset)
    print (f'Val. Loss: {loss}.')
    emulator.save_pretrained(os.path.join(args.save_model, f'checkpoint_{epoch}'))

torch.float32 float32 cuda
Creating features from dataset file at ../data/4_by_4_mult/train.txt
tgt_avg:  49.0
src_avg:  10.0
ratios:  0.20408163265306123
tgt_avg:  13.0
src_avg:  10.0
ratios:  0.7692307692307693
 1 3 3 8 * 5 1 0 5 <|endoftext|> 5 5 6 1 4 + 0 1 3 3 8 0 ( 5 6 9 4 2 1 ) + 0 0 0 0 0 0 0 ( 5 6 9 4 2 1 0 ) + 0 0 0 5 5 6 1 4 <|endoftext|> #### 5 6 9 9 7 7 1 4 <|endoftext|>
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 642, 642, 718, 352, 604, 1343, 657, 352, 513, 513, 807, 657, 357, 642, 718, 860, 604, 362, 352, 1267, 1343, 657, 657, 657, 657, 657, 657, 657, 357, 642, 718, 860, 604, 362, 352, 657, 1267, 1343, 657, 657, 657, 642, 642, 718, 352, 604, 220, 50256]
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 1303, 21017, 642, 718, 860, 860, 767, 767, 352, 604, 220, 50256]
[352, 513, 513, 807, 1635, 642, 352, 657, 642, 220, 50256, 1303, 21017, 642, 718, 860, 860, 767, 767, 352, 604, 220, 50256]
 1 3 3 8 * 5 1 0 5 <|endoftext|> #### 5 6 9

  0%|                                       | 3/50500 [00:00<1:42:18,  8.23it/s]

Step: 0. Loss: 6161.64599609375.


  0%|                                     | 103/50500 [00:07<1:01:52, 13.57it/s]

Step: 100. Loss: 638.393310546875.


...

Step: 50300. Loss: 50.696067810058594.


100%|██████████████████████████████████▉| 50403/50500 [1:02:53<00:07, 13.41it/s]

Step: 50400. Loss: 63.19522476196289.


100%|███████████████████████████████████████████| 63/63 [00:01<00:00, 38.44it/s]


Val. Loss: 64.931923828125.
Saving to ../train_models/4_by_4_mult/gpt2/emulator_initial/checkpoint_0


In [ ]:
import math
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
import argparse
import os
import inspect
import tqdm
import logging
import random
from itertools import chain

from data import CoTDataset, CoTDataCollator, extract_answer
from models.student import Student
from models.emulator import Emulator
from utils import get_sep_position

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
random.seed(1234)
torch.manual_seed(1234)
logging.disable(logging.WARNING) # disable WARNING, INFO and DEBUG logging everywhere

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


@torch.no_grad()
def evaluate(dataloader, tokenizer, ctx, emulator, student, max_new_tokens):
    total_instances = 0
    total_tokens = 0
    total_correct = 0
    total_correct_tokens = 0
    total_loss = 0
    for batch in tqdm.tqdm(dataloader):
        #import pdb; pdb.set_trace()
        input_ids_nocot = batch['input_ids_nocot'].to(device)
        labels_nocot = batch['labels_nocot'].to(device)
        batch_size = input_ids_nocot.shape[0]
        with ctx:
            emulated_teacher_states = emulator(input_ids=input_ids_nocot)
            outputs = student.compute_loss(input_ids=input_ids_nocot, labels=labels_nocot, teacher_states=emulated_teacher_states)
            loss = outputs.loss
            token_accuracy = outputs.token_accuracy.item()
        total_loss += outputs.total_loss.item()
        total_correct_tokens += outputs.total_correct.item()
        total_tokens += outputs.total_tokens
        total_instances += batch_size

        # Generate
        with ctx:
            beam_output = student.generate(
                input_ids=input_ids_nocot,
                teacher_states=emulated_teacher_states,
                max_new_tokens=max_new_tokens,
            )

        # Evaluate
        sep_positions = get_sep_position(input_ids_nocot, tokenizer.eos_token_id)
        for i, (input_ids_i, beam_output_i) in enumerate(zip(input_ids_nocot, beam_output)):
            sep_position = sep_positions[i].item()
            tgt = input_ids_i[sep_position+1:]
            tgt_text = tokenizer.decode(tgt, skip_special_tokens=True)
            ans = extract_answer(tgt_text)
            pred_text = tokenizer.decode(beam_output_i[0][sep_position+1:], skip_special_tokens=True)
            pred_ans = extract_answer(pred_text)
            if ans == pred_ans:
                total_correct += 1
            if i == 0:
                print (f'Input: {tokenizer.decode(input_ids_i[:sep_position], skip_special_tokens=True)}')
                print (f'Target: {tgt_text}')
                print (f'Predicted: {pred_text}')
                print ('')
    accuracy = total_correct / total_instances
    token_accuracy = total_correct_tokens / total_tokens
    loss = total_loss / total_tokens
    ppl = math.exp(loss)
    return accuracy, token_accuracy, ppl




In [ ]:
coupled_emulator_student_trainer_args=dict(
    debug=False,
    train_path="../data/4_by_4_mult/train.txt",
    val_path="../data/4_by_4_mult/valid.txt",
    teacher="../train_models/4_by_4_mult/gpt2/teacher/checkpoint_0",
    save_model="../train_models/4_by_4_mult/gpt2/emulator_initial",
    base_model='gpt2',
    epochs=1,
    batch_size=16,
    emulator="../train_models/4_by_4_mult/gpt2/teacher/checkpoint_0",
    student="../train_models/4_by_4_mult/gpt2/teacher/checkpoint_0",
    lr=5e-5,
    max_new_tokens=128,
    delta='dynamic',
    max_grad_norm=1.0,
    softmax_temperature=0.05,
    fix_emulator=False,
    
)

from types import SimpleNamespace

args = SimpleNamespace(**coupled_emulator_student_trainer_args)

In [ ]:

# parser = argparse.ArgumentParser()
# parser.add_argument('--emulator', type=str, required=True)
# parser.add_argument('--student', type=str, required=True)
# parser.add_argument('--train_path', type=str, required=True)
# parser.add_argument('--val_path', type=str, required=True)
# parser.add_argument('--save_model', type=str, required=True)
# parser.add_argument('--max_new_tokens', type=int, default=128)
# parser.add_argument('--epochs', type=int, default=5)
# parser.add_argument('--batch_size', type=int, default=32)
# parser.add_argument('--lr', type=float, default=5e-5)
# parser.add_argument('--max_grad_norm', type=float, default=1.0)
# parser.add_argument('--softmax_temperature', type=float, default=0.05)
# parser.add_argument('--fix_emulator', dest='fix_emulator', action='store_true')
# parser.set_defaults(fix_emulator=False)
# args = parser.parse_args()

# print (args)

dtype = 'float32'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ctx = torch.amp.autocast(device_type='cuda', dtype=ptdtype)
print (ptdtype, dtype, device)

# Load Student
student = Student.from_pretrained(args.student).to(device).to(ptdtype)

# Load Emulator
emulator = Emulator.from_pretrained(args.emulator).to(device).to(ptdtype)

# Load data
tokenizer = emulator.tokenizer
collate_fn = CoTDataCollator(tokenizer)
train_dataset = CoTDataset(tokenizer, args.train_path, 1024)
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, collate_fn=collate_fn, shuffle=True)
val_dataset = CoTDataset(tokenizer, args.val_path, 1024)
val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, collate_fn=collate_fn, shuffle=False)

# Create Optimizer
if args.fix_emulator:
    trainable_params = list(student.parameters())
    for p in emulator.parameters():
        p.requires_grad = False
else:
    trainable_params = list(student.parameters()) + list(emulator.parameters())
use_fused = 'fused' in inspect.signature(torch.optim.AdamW).parameters
extra_args = dict(fused=True) if use_fused else dict()
optimizer = torch.optim.AdamW(trainable_params, lr=args.lr, **extra_args)

emulator.eval() # to turn off dropout
student.eval() # to turn off dropout


# Train
step = 0
for epoch in range(args.epochs):
    print(f"Epoch {epoch}")

    for batch in tqdm.tqdm(train_dataloader):
        #import pdb; pdb.set_trace()
        input_ids_nocot = batch['input_ids_nocot'].to(device)
        labels_nocot = batch['labels_nocot'].to(device)
        with ctx:
            emulated_teacher_states = emulator(input_ids_nocot, requires_backward=not args.fix_emulator)
            outputs = student.compute_loss(input_ids=input_ids_nocot, labels=labels_nocot, teacher_states=emulated_teacher_states)
        loss = outputs.loss
        token_accuracy = outputs.token_accuracy.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(trainable_params, args.max_grad_norm)
        optimizer.step()
        optimizer.zero_grad()
        ppl = loss.exp().item()
        if step % 100 == 0:
            print (f"Step: {step}. PPL: {ppl}. Token Accuracy: {token_accuracy}")
        step += 1
    accuracy, token_accuracy, ppl = evaluate(val_dataloader, tokenizer, ctx, emulator, student, args.max_new_tokens)
    print (f'Val. PPL: {ppl}; Accuracy: {accuracy}; Token Accuracy: {token_accuracy}.')
    student.save_pretrained(os.path.join(args.save_model, 'student', f'checkpoint_{epoch}'))
    emulator.save_pretrained(os.path.join(args.save_model, 'emulator',  f'checkpoint_{epoch}'))